In [1]:
%load_ext sql
%sql sqlite:///PS2.db

'Connected: @PS2.db'

Problem Set #2
=======

### Deliverables:

Submit your answers using the `submission_template.txt` file that is posted on the class website. Follow the instructions on the file! Upload the file at Canvas (under PS2).


### Instructions / Notes:

**_Read these carefully_**

* You **may** create new IPython notebook cells to use for e.g. testing, debugging, exploring, etc.- this is encouraged in fact!- **just make sure that your final answer for each question is _in its own cell_ and _clearly indicated_**
* When you see `In [*]:` to the left of the cell you are executing, this means that the code / query is _running_.
    * **If the cell is hanging- i.e. running for too long: To restart the SQL connection, you must restart the entire python kernel**
    * To restart kernel using the menu bar: "Kernel >> Restart >> Clear all outputs & restart"), then re-execute the sql connection cell at top
    * You will also need to restart the connection if you want to load a different version of the database file
* Remember:
    * `%sql [SQL]` is for _single line_ SQL queries
    * `%%sql [SQL]` is for _multi line_ SQL queries
* _Have fun!_

Problem 1: Verifying Functional Dependencies [24 points]
---------

For this part, you will need to provide a _single_ SQL query which will check whether a certain condition holds on the **hospital** table in the provided database:

In [2]:
%sql select * from hospital limit 2;

 * sqlite:///PS2.db
Done.


provider,hospital,address,city,state,zip,county,phone_number,hospital_type,hospital_owner,emergency_service,condition,measure_code
10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Surgical Infection Prevention,SCIP-CARD-2
10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Surgical Infection Prevention,SCIP-INF-1


In [3]:
%sql PRAGMA table_info(hospital)

 * sqlite:///PS2.db
Done.


cid,name,type,notnull,dflt_value,pk
0,provider,integer,0,None,0
1,hospital,varchar(100),0,None,0
2,address,varchar(100),0,None,0
3,city,varchar(100),0,None,0
4,state,varchar(10),0,None,0
5,zip,integer,0,None,0
6,county,varchar(100),0,None,0
7,phone_number,integer,0,None,0
8,hospital_type,varchar(200),0,None,0
9,hospital_owner,varchar(400),0,None,0


In [11]:
%%sql
SELECT DISTINCT provider, hospital, address 
FROM hospital
LIMIT 3;

 * sqlite:///PS2.db
Done.


provider,hospital,address
10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD
10019,HELEN KELLER MEMORIAL HOSPITAL,1300 SOUTH MONTGOMERY AVENUE
10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE


You need to evaluate any requested conditions in the following way: **your query should return an empty result if and only if the condition holds on the instance.**  If the condition _doesn't hold_, your query should return something non-empty, but it doesn't matter what this is.

Note our language here: the conditions that we specify cannot be proved to hold **in general** without knowing the externally-defined functional dependencies; so what we mean is, _check whether they **are not violated** for the provided instance_.

You may assume that there are no `NULL` values in the tables.

### Part (a)  [14 points]

Is $\{provider\}$ a **superkey** for relation $Hospital$?

In [4]:
%%sql 
SELECT *
FROM hospital AS H1, hospital AS H2
WHERE H1.provider = H2.provider AND ((NOT(H1.hospital = H2.hospital)) OR (NOT(H1.address = H2.address))
                                                                      OR (NOT(H1.city = H2.city))
                                                                      OR (NOT(H1.state = H2.state))
                                                                      OR (NOT(H1.zip = H2.zip))
                                                                      OR (NOT(H1.county = H2.county))
                                                                      OR (NOT(H1.phone_number = H2.phone_number))
                                                                      OR (NOT(H1.hospital_type = H2.hospital_type))
                                                                      OR (NOT(H1.hospital_owner = H2.hospital_owner))
                                                                      OR (NOT(H1.emergency_service = H2.emergency_service))
                                                                      OR (NOT(H1.condition = H2.condition))
                                                                      OR (NOT(H1.measure_code = H2.measure_code)))
LIMIT 1;

 * sqlite:///PS2.db
Done.


provider,hospital,address,city,state,zip,county,phone_number,hospital_type,hospital_owner,emergency_service,condition,measure_code,provider_1,hospital_1,address_1,city_1,state_1,zip_1,county_1,phone_number_1,hospital_type_1,hospital_owner_1,emergency_service_1,condition_1,measure_code_1
10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Surgical Infection Prevention,SCIP-CARD-2,10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Heart Attack,AMI-1


In [26]:
%%sql
WITH T AS (SELECT count(DISTINCT(provider)) - count(*) AS A FROM hospital)
SELECT * FROM T WHERE NOT(T.A = 0);

 * sqlite:///PS2.db
(sqlite3.OperationalError) near "*": syntax error
[SQL: WITH T AS (SELECT count(DISTINCT(provider)) - count(DISTINCT *) AS A FROM hospital)
SELECT * FROM T WHERE NOT(T.A = 0);]
(Background on this error at: http://sqlalche.me/e/e3q8)


### Part (b) [10 points]

Does $\{Zip\} \rightarrow \{City, State\}$ hold for relation $Hospital$?

In [6]:
%%sql 
SELECT * 
FROM hospital AS H1, hospital AS H2
WHERE H1.zip = H2.zip AND ((NOT(H1.city = H2.city)) OR (NOT(H1.state = H2.state)))
limit 1;

 * sqlite:///PS2.db
Done.


provider,hospital,address,city,state,zip,county,phone_number,hospital_type,hospital_owner,emergency_service,condition,measure_code,provider_1,hospital_1,address_1,city_1,state_1,zip_1,county_1,phone_number_1,hospital_type_1,hospital_owner_1,emergency_service_1,condition_1,measure_code_1


Problem 2: Superkeys & Decompositions [40 points]
---------

Consider a relation $S(A,B,C,D,E,F)$ with the following functional dependencies:

* $\{A\} \rightarrow \{D\}$
* $\{A\} \rightarrow \{E\}$
* $\{D\} \rightarrow \{C\}$
* $\{D\} \rightarrow \{F\}$

In each part of this problem, we will examine different properties the provided schema.

To answer **yes**, provide python code that assigns the variable ```answer``` to ```True``` and assigns ```explanation``` to be a python string which contains a (short!) explanation of why.  For example:

```python
answer = True
explanation = "All keys are superkeys."
```

To answer **no**, provide python code that assigns the variable ```answer``` to ```False``` and assigns ```explanation``` to be a python string which contains a (short!) explanation of why.  For example:

```python
answer = False
explanation = "D is not a superkey because its closure is {D,C,F}."
```

### Part (a) [8 points]

Is it correct that ${A,B}$ is a superkey?

In [ ]:
answer = True
explanation "the closure of {A, B} = {A, B, C, D, E, F}, since A->D,E; D->C,F, so A->{C,D,E,F} and A,B->A,B,C,D,E,F"

### Part (b) [8 points]

Is it correct that the decomposition $ABC$, $CDE$, $EFA$ is lossless-join?

In [ ]:
answer = False
explanation "by applying chase algorithm, no row can be reduced without subscripts, first row = a,b,c,d1,e,f; second row = a2,b2,c,d,e,f2; third row: a,b3,c3,d1,e,f and cannoe be futher reduced"

### Part (c) [8 points]

Is it correct that the decomposition $ABC$, $CDE$, $EFA$ is dependency preserving?

In [ ]:
answer = False
explanation "{𝐴}→{𝐷} is missing and cannot be enforced"

### Part (d) [8 points]

Is the functional dependency $\{A\} \rightarrow \{C,F\}$ logically implied by FDs present in the relation?

In [ ]:
answer = True
explanation "A->D, D->C, D->F, so that A->C,F by armstrong axioms"

### Part (e) [8 points]

Is it correct that relation $S$ is in BCNF? 

In [23]:
answer = False
explanation "A->D is a nontrivial FD but A is not a superkey, since A->B is false"

Problem 3: Relational Algebra [36 points]
---------

Consider the following relational schema:
*  `JournalArticle(articleID, title, journal, volume, year, month)`
*  `ConferenceArticle(articleID, title, conference, year, location)`
*  `Person(name, affiliation)`
*  `Author(name, articleID)`

Express the following queries in the extended Relational Algebra (you can also use the aggregation operator if necessary). To write the RA expression, use the LaTex mode that ipython notebook provides. For example:

$$\pi_{name}(\sigma_{affiliation="UW-Madison"}(Person))$$ 

### Part (a) [9 points]

Output the names of everyone who is affiliated with `UW-Madison` and who has submitted during 2018 at least one journal article, but no conference articles:

$$\pi_{name}(\sigma_{affiliation="UW-Madison"}(Person) \bowtie (Author \bowtie (\pi_{articleID}(\sigma_{year=2018}(JournalArticle)) - \pi_{articleID}(\sigma_{year=2018}(ConferenceArticle)))))$$ 

### Part (b) [9 points]

Output the names of the people who coauthored an article (in a journal or conference) with `John Doe`. Be careful: a person cannot be coauthor with herself!

$$\pi_{name}(\sigma_{name="John Doe"}(Author) \bowtie \pi_{articleID}(\sigma_{NOT name="John Doe"}(Author)))$$ 

### Part (c) [9 points]

Count how many journal *or* conference articles were published in 2018 by `John Doe`.

$$\gamma_{COUNT(articleID)}(\pi_{artileID}(\sigma_{name = "John Doe"}(Author) \bowtie \sigma_{year=2018}(ConferenceArticle)) \cup \pi_{artileID}(\sigma_{name = "John Doe"}(Author) \bowtie \sigma_{year=2018}(JournalArticle)))$$ 

### Part (d) [9 points]

Output the names of everyone who published at least as many conference as journal papers during every year in the span from 2000 to 2018 (including 2000 and 2018). 

$$LET A = \rho_{name, year, numOfJA}(\gamma_{name, year, COUNT(articleID)}(Author \bowtie \sigma_{year<=2018 \wedge year>=2000}(JournalArticle)))$$
$$B = \rho_{name, year, numOfCA}(\gamma_{name, year, COUNT(articleID)}(Author \bowtie \sigma_{year<=2018 \wedge year>=2000}(ConferenceArticle)))$$
$$answer = \pi_{name}(Person) - \pi_{name}(\sigma_{numOfCA < numOfJA}(A \bowtie B)) - \pi_{name}(\pi_{name, year}(B)- \pi_{name, year}(A))$$